# Courmayeur - Champex - Chamonix Average Pace Prediction

All runners who
have ran the CCC and then went on to run the UTMB are also found. Each record contains the
individual stage paces and average pace of the non PB run and the PB run (or CCC run and the
UTMB run in the other case). These times are weighted according to their similarity to the query
runner (most similar gets a higher weight), to see if better predictions can be made.


In [38]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

In [39]:
# Constants
pace_threshold = 1

## Read in Scraped CCC Data

In [40]:
# Read in Split Timings
df = pd.read_csv('../../data/CCC_1km_splits.csv', encoding = "ISO-8859-1")

In [41]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Year,bib,name_mask,gender,category,rank,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,0,NaN,2016,3002,1eeca1e237fb722bee11d6b3a89182cf,Male,Senior Men (23-39),NaN,NaN,8.326923,4.449612,5.405405,4.506410,NaN,10.874074,NaN,4.446667,NaN,6.095238,NaN,NaN,NaN,8.436578,NaN,NaN,6.081633,11.584906,NaN,NaN,5.625000,NaN,12.222944,NaN,8.888889,NaN,NaN,5.193694,NaN,730.066667,7.295569
1,1,NaN,2016,3003,79c472880467010197a2e94fbb718b52,Male,Senior Men (23-39),NaN,NaN,8.193910,4.705426,5.454955,4.487179,NaN,11.651852,NaN,4.666667,NaN,6.975000,NaN,NaN,NaN,8.134218,NaN,NaN,5.173469,11.893082,NaN,NaN,6.383333,NaN,11.344156,NaN,7.828283,NaN,NaN,5.087838,NaN,735.333333,7.284241
2,2,NaN,2016,3012,18896d8a594562fbc883979eb5985bd3,Male,Masters Men 1 (40-49),3.0,NaN,8.485577,4.674419,5.777027,4.897436,NaN,12.962963,NaN,5.046667,NaN,6.615476,NaN,NaN,NaN,8.569322,NaN,NaN,5.125850,11.748428,NaN,NaN,4.858333,NaN,10.699134,NaN,8.000000,NaN,NaN,5.299550,NaN,739.350000,7.340013
3,3,3.0,2016,3005,78612f155eb4cf9201cf5802867d8119,Male,Senior Men (23-39),4.0,NaN,8.700321,4.864341,5.691441,4.891026,NaN,12.211111,NaN,4.886667,NaN,6.463095,NaN,NaN,NaN,8.741888,NaN,NaN,5.489796,12.361635,NaN,NaN,4.972222,NaN,11.023810,NaN,7.363636,NaN,NaN,5.680180,NaN,745.500000,7.381512
4,4,4.0,2016,3015,28fe5c601616ac3c31f9af34d49d6e71,Male,Senior Men (23-39),5.0,NaN,8.860577,4.755814,5.695946,5.317308,NaN,14.270370,NaN,4.818333,NaN,6.453571,NaN,NaN,NaN,8.556047,NaN,NaN,5.996599,12.185535,NaN,NaN,5.230556,NaN,11.562771,NaN,9.090909,NaN,NaN,6.211712,NaN,772.183333,7.786146


In [42]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib', 'rank'], axis = 1)

In [43]:
df.shape

(20471, 36)

In [44]:
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,2016,1eeca1e237fb722bee11d6b3a89182cf,Male,Senior Men (23-39),NaN,8.326923,4.449612,5.405405,4.506410,NaN,10.874074,NaN,4.446667,NaN,6.095238,NaN,NaN,NaN,8.436578,NaN,NaN,6.081633,11.584906,NaN,NaN,5.625000,NaN,12.222944,NaN,8.888889,NaN,NaN,5.193694,NaN,730.066667,7.295569
1,2016,79c472880467010197a2e94fbb718b52,Male,Senior Men (23-39),NaN,8.193910,4.705426,5.454955,4.487179,NaN,11.651852,NaN,4.666667,NaN,6.975000,NaN,NaN,NaN,8.134218,NaN,NaN,5.173469,11.893082,NaN,NaN,6.383333,NaN,11.344156,NaN,7.828283,NaN,NaN,5.087838,NaN,735.333333,7.284241
2,2016,18896d8a594562fbc883979eb5985bd3,Male,Masters Men 1 (40-49),NaN,8.485577,4.674419,5.777027,4.897436,NaN,12.962963,NaN,5.046667,NaN,6.615476,NaN,NaN,NaN,8.569322,NaN,NaN,5.125850,11.748428,NaN,NaN,4.858333,NaN,10.699134,NaN,8.000000,NaN,NaN,5.299550,NaN,739.350000,7.340013
3,2016,78612f155eb4cf9201cf5802867d8119,Male,Senior Men (23-39),NaN,8.700321,4.864341,5.691441,4.891026,NaN,12.211111,NaN,4.886667,NaN,6.463095,NaN,NaN,NaN,8.741888,NaN,NaN,5.489796,12.361635,NaN,NaN,4.972222,NaN,11.023810,NaN,7.363636,NaN,NaN,5.680180,NaN,745.500000,7.381512
4,2016,28fe5c601616ac3c31f9af34d49d6e71,Male,Senior Men (23-39),NaN,8.860577,4.755814,5.695946,5.317308,NaN,14.270370,NaN,4.818333,NaN,6.453571,NaN,NaN,NaN,8.556047,NaN,NaN,5.996599,12.185535,NaN,NaN,5.230556,NaN,11.562771,NaN,9.090909,NaN,NaN,6.211712,NaN,772.183333,7.786146


In [45]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
15085,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951
17920,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072
8449,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
18728,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
8339,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679


# UTMB

In [46]:
# Read in Split Timings
df1 = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [47]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,Year,bib,name_mask,team,gender,category,rank,nationality,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,0,NaN,2017,4.0,7efc0cb7e70d7cd9da46a58bbf01af66,Salomon,Male,Senior Men (23-39),NaN,FR,5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,1,NaN,2017,NaN,1e4ba9a22b3fecb9136ecfcf7f470868,Salomon,Male,Senior Men (23-39),NaN,ES,5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2,NaN,2017,14.0,0d928f7482046900e19dba217d7befa5,Hoka,Male,Senior Men (23-39),3.0,US,5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,3,3.0,2017,7.0,93f20063908ba5d48724dcf31740a52a,Asics,Male,Senior Men (23-39),4.0,FR,5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,4,4.0,2017,NaN,7479fb90beffad048c5f772ba1915b4e,Hoka,Male,Senior Men (23-39),5.0,US,5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [48]:
# Drop unnecessary columns
df1 = df1.drop(['Unnamed: 0', 'Unnamed: 0.1', 'team', 'nationality', 'bib', 'rank', 'gender'], axis = 1)

In [49]:
df1.head()

,Year,name_mask,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [50]:
# Filter out NaN times
df1 = df1.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df1.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df1.head()

,Year,name_mask,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# With Personal Best Time

In [53]:
# Get Personal Best
pb = df1[df1.time.groupby(df1.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb_y = pb.copy()
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
#ccc = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [54]:
pb.head()

,name_mask,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,0001fff82652ce3818504f26771297a3,7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,0002b9e98b5761bb4aa0960616f45ca6,8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,000bf78327bdfa8c6298fc9a76921305,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,000d01e06e9abfe51cdb08ff2567f125,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,00156070bc3ed3fd8b6b87dabd51c2fc,NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


In [55]:
name = pb['name_mask']

In [56]:
name.head()

11228    0001fff82652ce3818504f26771297a3
3623     0002b9e98b5761bb4aa0960616f45ca6
5634     000bf78327bdfa8c6298fc9a76921305
7880     000d01e06e9abfe51cdb08ff2567f125
15238    00156070bc3ed3fd8b6b87dabd51c2fc
Name: name_mask, dtype: object

In [57]:
pb = pb.add_suffix('_pb')

In [58]:
pb = pb.drop(['name_mask_pb'], axis=1)

In [59]:
pb = pb.join(name)

In [60]:
pb

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,name_mask
11228,7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557,0001fff82652ce3818504f26771297a3
3623,8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491,0002b9e98b5761bb4aa0960616f45ca6
5634,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938,000bf78327bdfa8c6298fc9a76921305
7880,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371,000d01e06e9abfe51cdb08ff2567f125
15238,NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108,00156070bc3ed3fd8b6b87dabd51c2fc
8786,8.161765,NaN,NaN,8.004386,NaN,10.145631,12.958848,18.939394,10.480392,14.407767,10.850340,20.237500,10.292593,14.182927,NaN,NaN,27.693878,12.916667,15.291667,23.518519,12.421667,13.336905,NaN,NaN,23.163717,NaN,NaN,19.513605,25.518868,NaN,18.911111,NaN,25.900433,NaN,NaN,21.712121,NaN,NaN,13.033784,2627.083333,1415.350000,16.316437,001e405237215c01ebb54168bc0d93ec
15711,NaN,NaN,7.596899,NaN,NaN,9.894822,13.777778,18.745455,10.346405,16.810680,8.343537,19.191667,8.600000,12.126016,NaN,NaN,25.476190,12.045045,12.076923,23.107407,9.490000,12.673810,21.461279,NaN,NaN,16.722222,NaN,NaN,30.515723,NaN,20.141667,NaN,27.378788,NaN,NaN,15.545455,NaN,NaN,12.918919,2523.133333,1229.833333,15.868986,0027471ab21ee13fa15229849e7deebb
10507,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501,003cc0338235bb1862eb7b9bdd31ec9c
3940,9.375000,NaN,NaN,8.

In [62]:
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
15085,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951
17920,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072
8449,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
18728,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
8339,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679


In [63]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = df.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [64]:
name_y = pb_y['name_mask']

pb_y = pb_y.add_suffix('_pby')

pb_y = pb_y.drop(['name_mask_pby'], axis=1)

pb_y = pb_y.join(name)

# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge_y = df.merge(pb_y,how='left',on='name_mask',suffixes=('', '_pby'))

In [65]:
ccc_first = merge_y[merge_y.Year < merge_y.Year_pby]

In [66]:
len(ccc_first)

2527

In [67]:
#merge = merge.groupby('name_mask').head(1)

In [68]:
merge.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
3,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
4,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,

In [69]:
first_df = len(merge)

In [73]:
first_df

13983

In [148]:
merge = merge.dropna(subset=['time_pb'])

In [149]:
merge.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
2,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
3,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
4,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
27,2007,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),12.030612,NaN,NaN,8.445946,8.253205,NaN,17.918519,55.873874,NaN,NaN,NaN,NaN,15.240741,NaN,NaN,10.634921,NaN,NaN,NaN,NaN,NaN,26.361111,NaN,NaN,13.309524,NaN,NaN,12.264368,NaN,NaN,1018.333333,18.033282,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,Na

In [150]:
merge = merge[merge.name_mask != '17de1d15371b39ce70423fabda7d5370']
merge = merge[merge.name_mask != '379390d8a2fdd5429f7a0945e6d0f1ed']
merge = merge[merge.name_mask != '5589d307926d2732570fd1920263ec9c']
merge = merge[merge.name_mask != '75999421377de5d9d4e1a8288f79dd42']
merge = merge[merge.name_mask != 'ca2448a548fa4a95f279015dc3081717']

In [151]:
merge.iloc[[2268]]

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
11120,2015,ca328535829aa215f7d773ee1ebe3ae8,Male,Masters Men 2 (50-59),NaN,11.876603,7.410853,8.027027,NaN,22.330128,NaN,NaN,7.123333,NaN,8.760714,NaN,NaN,NaN,12.687316,NaN,NaN,8.010204,16.371069,NaN,NaN,7.805556,NaN,15.575758,NaN,12.025253,NaN,NaN,8.421171,NaN,1062.933333,11.26346,9.301471,NaN,NaN,8.467105,NaN,10.498382,13.771605,18.981818,11.232026,18.498382,9.459184,18.508333,10.259259,13.699187,NaN,NaN,31.612245,13.648649,16.076923,24.837037,11.295,15.065476,NaN,NaN,23.287611,NaN,NaN,16.35034,NaN,25.434028,NaN,18.25,NaN,NaN,25.040541,NaN,NaN,20.619048,14.184685,2673.75,1531.85,16.599097


In [152]:
merge = merge.reset_index(drop=True)

In [153]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [154]:
comb

,category,gender
0,Masters Men 1 (40-49),Male
1,Senior Men (23-39),Male
2,Senior Men (23-39),Male
3,Senior Men (23-39),Male
4,Senior Men (23-39),Male
5,Masters Men 1 (40-49),Male
6,Senior Men (23-39),Male
7,Masters Men 1 (40-49),Male
8,Masters Men 1 (40-49),Male
9,Senior Men (23-39),Male


In [155]:
merge = merge.reset_index(drop=True)

In [156]:
merge.shape

(2798, 78)

In [157]:
# Update pb and ccc to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
ccc = merge.loc[:, ~merge.columns.str.contains('pb')]

In [158]:
ccc.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
1,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
2,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679
3,2007,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),12.030612,NaN,NaN,8.445946,8.253205,NaN,17.918519,55.873874,NaN,NaN,NaN,NaN,15.240741,NaN,NaN,10.634921,NaN,NaN,NaN,NaN,NaN,26.361111,NaN,NaN,13.309524,NaN,NaN,12.264368,NaN,NaN,1018.333333,18.033282
4,2007,00791e76e3cd48d47e4dc93d7b60ef95,Male,Senior Men (23-39),14.455782,NaN,NaN,9.018018,8.737179,NaN,15.574074,53.292793,NaN,NaN,NaN,NaN,15.873737,NaN,NaN,10.748677,NaN,NaN,NaN,NaN,NaN,25.047222,NaN,NaN,10.939153,NaN,NaN,9.818008,NaN,NaN,979.750000,17.350464


In [159]:
pb.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
1,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
2,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
3,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
4,8.310049,NaN,NaN,6.526316,NaN,9.925566,10.002058,15.633333,8.653595,13.300971,8.272109,17.025000,9.625926,11.991870,NaN,NaN,24.132653,10.704955,12.419872,17.788889,10.191667,11.866667,16.170034,NaN,NaN,14.177249,NaN,NaN,19.396226,NaN,9.777778,NaN,16.153680,NaN,NaN,13.343434,NaN,NaN,8.702703,2067.933333,832.983333,12.670525


# Define Functions

In [160]:
def find_similar_times(index):
    query = ccc.iloc[index]
    same_category = ccc[ccc.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(279).head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
10,2008,00b1970576a81a14d9b98c22cc3fd95a,Male,Senior Men (23-39),20.557823,NaN,NaN,12.849099,11.826923,NaN,19.025926,NaN,7.673333,NaN,10.165476,NaN,17.932660,NaN,NaN,12.584656,NaN,NaN,21.902516,NaN,NaN,11.088889,NaN,20.183983,NaN,11.540404,NaN,NaN,7.270270,NaN,1248.716667,14.200151
15,2009,0127b3f1d01a9ca57b3b41c106e5ab14,Male,Senior Men (23-39),26.568027,NaN,NaN,15.146396,11.054487,NaN,16.022222,NaN,6.233333,NaN,8.673810,NaN,13.441077,NaN,NaN,10.529101,NaN,NaN,20.059748,NaN,NaN,10.780556,NaN,21.740260,NaN,15.424242,NaN,NaN,13.078829,NaN,1241.100000,14.519391
22,2008,02118c6ec11c23c75718d7c067d3f742,Male,Senior Men (23-39),25.751701,NaN,NaN,17.795045,11.378205,NaN,18.803704,NaN,9.533333,NaN,11.167857,NaN,15.929293,NaN,NaN,14.402116,NaN,NaN,18.066038,NaN,NaN,12.950000,NaN,18.491342,NaN,13.853535,NaN,NaN,11.587838,NaN,1349.066667,15.362308
23,2010,023556ffdab2dc989ddef50ace0a63c5,Male,Senior Men (23-39),NaN,15.834936,12.333333,12.313063,7.801282,NaN,14.725926,NaN,5.730000,NaN,7.338095,NaN,13.303030,NaN,NaN,8.830688,NaN,NaN,14.795597,NaN,NaN,8.505556,NaN,NaN,NaN,54.520202,NaN,NaN,12.144144,NaN,1056.200000,14.475066
25,2014,02506d080decc6dedc59fb5c1e10387a,Male,Senior Men (23-39),NaN,10.447115,6.011628,7.054054,7.086538,NaN,19.211111,NaN,9.526667,NaN,21.513095,NaN,NaN,NaN,18.635693,NaN,NaN,11.62585,23.808176,NaN,NaN,10.636111,NaN,23.673160,NaN,17.101010,NaN,NaN,8.885135,NaN,1468.466667,13.943953


In [162]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(30)

2296    0.999459
2192    0.999446
2334    0.999190
2213    0.999089
1705    0.998855
723     0.998586
1533    0.998230
479     0.998099
1051    0.998053
306     0.997992
dtype: float64

In [164]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(279)

399     1.144035
2138    1.610735
535     1.647605
154     1.721844
321     1.767122
1417    1.829074
2311    2.033087
2086    2.044031
571     2.046689
966     2.124474
dtype: float64

In [165]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(30)

2296    0.100076
2192    0.100075
2334    0.100049
2213    0.100039
1705    0.100015
723     0.099989
1533    0.099953
479     0.099940
1051    0.099935
306     0.099929
dtype: float64

In [166]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(279)

399     0.104037
2138    0.101151
535     0.100923
154     0.100464
321     0.100184
1417    0.099801
2311    0.098539
2086    0.098472
571     0.098455
966     0.097974
dtype: float64

In [167]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(279)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
782,2006,45c7b06cca450b7eaeccdd4b1348c20f,Male,Senior Men (23-39),11.282313,NaN,NaN,7.610360,NaN,22.403846,NaN,NaN,7.253333,7.092157,NaN,12.020115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.604167,NaN,NaN,NaN,13.820106,NaN,NaN,13.162835,NaN,NaN,1033.700000,14.805470,NaN,4.735507,NaN,NaN,8.294444,8.210356,10.895062,15.218182,9.954248,12.325243,9.357143,16.683333,NaN,20.247967,NaN,NaN,20.462585,10.968468,11.471154,18.696296,10.331667,12.275000,17.745791,NaN,NaN,18.129630,NaN,NaN,20.270440,NaN,20.150000,NaN,24.919913,NaN,NaN,23.479798,NaN,NaN,20.126126,2325.433333,1068.450000,14.997755,0.100061,1.500686
2742,2006,fa55690d12ea22ede158a4a472ae6816,Male,Senior Men (23-39),11.897959,NaN,NaN,7.734234,NaN,21.544872,NaN,NaN,7.371667,7.329412,NaN,11.893678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.527244,NaN,NaN,NaN,12.325397,NaN,NaN,11.486590,NaN,NaN,1000.433333,14.345672,NaN,NaN,7.188372,NaN,NaN,8.234628,11.495885,16.521212,9.153595,13.741100,8.904762,17.937500,8.396296,10.426829,NaN,NaN,26.510204,11.193694,10.567308,20.400000,10.596667,12.661905,15.164983,NaN,NaN,14.476190,NaN,NaN,21.066038,NaN,13.763889,NaN,20.244589,NaN,NaN,15.247475,NaN,NaN,11.639640,2178.100000,884.800000,13.718816,0.100061,1.372714
966,2006,57917da5727da289359c504ff94f505f,Male,Senior Men (23-39),12.142857,NaN,NaN,8.101351,NaN,23.980769,NaN,NaN,7.255000,6.790196,NaN,11.543103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.645833,NaN,NaN,NaN,14.330688,NaN,NaN,11.994253,NaN,NaN,1027.516667,14.864895,NaN,NaN,NaN,NaN,31.052778,11.003236,15.131687,18.166667,10.506536,16.014563,10.030612,18.295833,12.374074,10.508130,NaN,NaN,27.965986,15.132883,16.240385,26.162963,11.141667,15.215476,22.378788,NaN,NaN,15.902116,NaN,NaN,24.462264,NaN,11.622222,NaN,18.454545,NaN,NaN,14.333333,NaN,NaN,8.815315,2513.150000,1256.166667,16.561394,0.100045,1.656885
154,2006,0e186d6235074e96b7328288cee347bc,Male,Senior Men (23-39),13.952381,NaN,NaN,8.409910,NaN,23.157051,NaN,NaN,7.521667,6.970588,NaN,10.686782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.641026,NaN,NaN,NaN,11.042328,NaN,NaN,11.049808,NaN,NaN,994.233333,14.381282,NaN,NaN,9.117054,NaN,NaN,9.901294,13.452675,19.596970,11.114379,16.352751,9.258503,17.675000,9.355556,12.357724,NaN,NaN,22.778912,13.959459,12.945513,20.829630,10.475000,12.046429,19.698653,NaN,Na

In [168]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(279)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
399,2006,2306a99302f8a71ee6e9a0da10573cbc,Male,Senior Men (23-39),11.122449,NaN,NaN,8.351351,NaN,21.506410,NaN,NaN,8.790000,8.713725,NaN,11.290230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.092949,NaN,NaN,NaN,16.092593,NaN,NaN,14.323755,NaN,NaN,1108.950000,15.698162,NaN,NaN,NaN,NaN,NaN,28.736246,15.989712,18.369697,12.117647,15.742718,9.598639,17.700000,9.903704,11.235772,NaN,NaN,29.204082,12.736486,16.849359,25.207407,12.023333,16.625000,34.621212,NaN,NaN,17.941799,NaN,NaN,20.273585,NaN,10.025000,NaN,18.489177,NaN,NaN,15.727273,NaN,NaN,10.295045,2644.166667,1387.183333,17.246041,0.104037,1.794224
2138,2006,c023193c381835f647be9f0d2a50bf6d,Male,Senior Men (23-39),13.285714,NaN,NaN,8.144144,NaN,25.038462,NaN,NaN,7.980000,7.625490,NaN,12.227011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.943910,NaN,NaN,NaN,13.621693,NaN,NaN,11.159004,NaN,NaN,1069.416667,15.447270,NaN,5.326087,NaN,NaN,8.463889,9.368932,13.022634,16.427273,9.098039,14.279935,9.561224,18.516667,8.262963,9.983740,NaN,NaN,28.374150,10.759009,12.041667,21.277778,9.948333,10.970238,20.742424,NaN,NaN,12.931217,NaN,NaN,21.267296,NaN,12.122222,NaN,21.344156,NaN,NaN,11.858586,NaN,NaN,8.945946,2230.166667,973.183333,13.537267,0.101151,1.369308
535,2006,2f2e48c3dd8a4abd344726ee22268c67,Male,Senior Men (23-39),13.438776,NaN,NaN,8.657658,NaN,22.112179,NaN,NaN,7.440000,6.996078,NaN,11.060345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.653846,NaN,NaN,NaN,10.944444,NaN,NaN,9.885057,NaN,NaN,979.066667,14.132043,NaN,NaN,8.997674,NaN,NaN,10.061489,13.152263,18.600000,13.049020,15.147249,10.166667,19.275000,10.214815,13.394309,NaN,NaN,25.673469,13.400901,14.419872,27.188889,11.200000,14.416667,23.809764,NaN,NaN,18.328042,NaN,NaN,24.556604,NaN,12.250000,NaN,21.203463,NaN,NaN,14.525253,NaN,NaN,12.403153,2550.233333,1256.933333,15.888459,0.100923,1.603511
154,2006,0e186d6235074e96b7328288cee347bc,Male,Senior Men (23-39),13.952381,NaN,NaN,8.409910,NaN,23.157051,NaN,NaN,7.521667,6.970588,NaN,10.686782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.641026,NaN,NaN,NaN,11.042328,NaN,NaN,11.049808,NaN,NaN,994.233333,14.381282,NaN,NaN,9.117054,NaN,NaN,9.901294,13.452675,19.596970,11.114379,16.352751,9.258503,17.675000,9.355556,12.357724,NaN,NaN,22.778912,13.959459,12.945513,20.829630,10.475000,12.046429,19.698653,NaN,NaN,1

In [169]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]

    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(5)

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
531,2016,2f157fc62c573592f65d2a256f7ef43b,Male,Masters Men 1 (40-49),NaN,12.887821,7.046512,8.132883,7.182692,NaN,17.462963,NaN,6.803333,NaN,9.351190,NaN,NaN,NaN,12.709440,NaN,NaN,9.459184,17.191824,NaN,NaN,7.994444,NaN,14.701299,NaN,11.838384,NaN,NaN,8.101351,NaN,1080.666667,10.775951,7.351716,NaN,NaN,5.962719,NaN,8.043689,10.209877,15.042424,7.745098,11.855987,7.146259,15.566667,7.714815,9.849593,NaN,NaN,20.363946,10.229730,11.115385,21.740741,8.105000,10.858333,NaN,NaN,15.277286,NaN,NaN,11.153061,19.003145,NaN,12.080556,NaN,22.194805,NaN,NaN,19.777778,NaN,NaN,12.628378,2021.866667,810.133333,12.542374
1833,2015,a240e2fdbc243de060f4901ff9ccaa29,Male,Masters Men 1 (40-49),NaN,15.310897,8.717054,8.439189,NaN,22.769231,NaN,NaN,7.186667,NaN,9.503571,NaN,NaN,NaN,10.915929,NaN,NaN,7.340136,19.569182,NaN,NaN,7.472222,NaN,13.768398,NaN,11.368687,NaN,NaN,6.268018,NaN,1080.266667,11.433014,9.776961,NaN,NaN,7.982456,NaN,13.236246,13.312757,19.478788,11.624183,15.600324,11.129252,22.741667,11.166667,13.430894,NaN,NaN,32.619048,13.506757,15.964744,23.759259,12.388333,15.482143,19.031987,NaN,NaN,19.018519,NaN,NaN,28.433962,NaN,9.802778,NaN,22.675325,NaN,NaN,19.848485,NaN,NaN,14.506757,2671.266667,1436.316667,16.521595
1828,2015,a1f925069cd95ebb6e202c8d17bafd7e,Male,Masters Men 1 (40-49),NaN,10.738782,6.449612,7.740991,NaN,22.253205,NaN,NaN,6.491667,NaN,8.386905,NaN,NaN,NaN,11.387906,NaN,NaN,7.289116,16.827044,NaN,NaN,7.366667,NaN,16.207792,NaN,12.202020,NaN,NaN,6.434685,NaN,1005.216667,10.752030,8.752451,NaN,NaN,7.774123,NaN,8.726537,13.331276,16.460606,8.591503,12.817152,8.061224,13.762500,11.566667,10.747967,NaN,NaN,23.452381,11.006757,11.346154,21.292593,11.926667,12.696429,NaN,15.201946,NaN,NaN,22.903846,NaN,18.273585,NaN,9.138889,NaN,NaN,9.806878,NaN,NaN,11.296935,NaN,NaN,2179.883333,943.166667,12.997177
2232,2016,c6fb3fdfa87477e04e727d02333fc9ee,Male,Masters Men 1 (40-49),NaN,11.466346,6.674419,7.716216,7.871795,NaN,17.229630,NaN,6.705000,NaN,9.646429,NaN,NaN,NaN,13.005900,NaN,NaN,7.578231,17.987421,NaN,NaN,7.219444,NaN,15.876623,NaN,11.691919,NaN,NaN,6.905405,NaN,1060.300000,10.541056,8.208333,NaN,NaN,5.736842,NaN,8.276699,NaN,28.130303,8.241830,11.909385,7.581633,14.650000,8.340741,9.796748,NaN,NaN,24.717687,9.006757,10.253205,15.829630,7.81000

In [170]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


16.008372374941608

In [171]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


14.859204068259578

In [172]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.316688474550597

In [173]:
def get_best_ccc(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_best_ccc(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


14.075860148863466

In [174]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


14.464337182305984

In [175]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_ccc(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(680)
get_prediction(680, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.396301656363686

# Calculate Predictions

In [176]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [177]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,11.246015,15.222938,11.995858,14.050342,14.050399,14.394697
1,21.929566,15.222938,15.726010,16.750925,16.750918,16.442480
2,10.436679,12.416371,11.363543,13.571467,13.571714,13.875511
3,18.033282,13.945114,14.328692,16.382603,16.382770,15.795056
4,17.350464,12.670525,14.945903,16.093129,16.093250,15.758484
5,11.414342,15.628671,13.055773,13.866630,13.866438,13.324389
6,17.027937,16.952247,14.429175,15.989975,15.990421,16.126535
7,10.836450,14.358531,12.474714,14.789060,14.789291,14.152184
8,11.295707,14.358531,12.048864,14.285569,14.285878,14.600266
9,10.192421,16.232838,10.660584,12.280184,12.280028,13.775937


In [178]:
df = comb.join(df)

In [179]:
df.to_csv('../../data/CCC_predictions.csv')

In [207]:
df.head()

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Masters Men 1 (40-49),Male,11.246015,15.222938,11.995858,14.050342,14.050399,14.394697
1,Senior Men (23-39),Male,21.929566,15.222938,15.726010,16.750925,16.750918,16.442480
2,Senior Men (23-39),Male,10.436679,12.416371,11.363543,13.571467,13.571714,13.875511
3,Senior Men (23-39),Male,18.033282,13.945114,14.328692,16.382603,16.382770,15.795056
4,Senior Men (23-39),Male,17.350464,12.670525,14.945903,16.093129,16.093250,15.758484


In [35]:
predict = pd.read_csv('../../data/CCC_predictions.csv')

In [37]:
predict = len(predict)

In [72]:
first_df

13983

In [70]:
predict / first_df

0.20010012157619966

#  Best Pace row

In [180]:
def best_pace_rec(index):
    query = ccc.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.sort_values(by='AVG_Pace_pb', ascending=True)
    row_pace = row_pace.head(1)
   # row_pace = pd.DataFrame(row_pace)
   # row_pace = row_pace.T
    row_pace = row_pace.filter(like='to')
    row_pace.index = query.index

    return row_pace


best_pace_rec(5)

,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
5,NaN,10.608974,7.275194,7.436937,7.358974,NaN,18.148148,NaN,6.42,NaN,9.05119,NaN,NaN,NaN,13.283186,NaN,NaN,9.088435,17.157233,NaN,NaN,8.325,NaN,16.5671,NaN,13.570707,NaN,NaN,7.204955,NaN,6.960784,NaN,NaN,5.620614,NaN,8.875405,10.547325,14.160606,7.957516,12.131068,7.894558,17.570833,8.307407,10.878049,NaN,NaN,24.94898,9.903153,10.458333,17.37037,9.038333,10.446429,15.082492,NaN,NaN,10.880952,NaN,NaN,19.128931,NaN,8.686111,NaN,16.982684,NaN,NaN,11.616162,NaN,NaN,10.518018


In [184]:
pb1 = pb.filter(like='to')

In [185]:
pb1.shape

(2798, 39)

In [186]:
pb_top = pd.DataFrame()

In [187]:
pb_top

""


In [188]:
for i in range(pb1.shape[0]):
    pb_top = pb_top.append(best_pace_rec(i), ignore_index=True)

In [189]:
pb_top

,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,NaN,10.019231,6.465116,6.504505,NaN,20.073718,NaN,NaN,6.145000,NaN,8.846429,NaN,NaN,NaN,12.353982,NaN,NaN,7.394558,16.251572,NaN,NaN,8.361111,NaN,16.926407,NaN,10.257576,NaN,NaN,6.599099,NaN,6.627451,NaN,NaN,5.456140,NaN,7.582524,11.248971,14.518182,7.503268,11.203883,15.721088,12.900000,7.118519,9.581301,23.611111,NaN,14.989796,9.288288,8.631410,15.437037,6.750000,9.766667,NaN,NaN,13.104720,NaN,NaN,7.914966,17.150943,NaN,7.433333,NaN,15.958874,NaN,NaN,11.646465,NaN,NaN,8.238739
1,18.510204,NaN,NaN,11.418919,10.887821,NaN,19.544444,71.972973,NaN,NaN,NaN,NaN,18.698653,NaN,NaN,14.894180,NaN,NaN,NaN,NaN,NaN,31.325000,NaN,NaN,11.539683,NaN,NaN,12.436782,NaN,NaN,NaN,NaN,8.427907,NaN,NaN,10.598706,13.681070,18.960606,12.375817,14.624595,8.982993,17.462500,9.592593,12.430894,NaN,NaN,26.792517,12.295045,13.035256,21.592593,10.710000,13.915476,19.146465,NaN,NaN,17.150794,NaN,NaN,23.805031,NaN,13.816667,NaN,22.911255,NaN,NaN,12.661616,NaN,NaN,10.245495
2,NaN,10.910256,6.391473,7.560811,NaN,22.368590,NaN,NaN,6.518333,NaN,9.108333,NaN,NaN,NaN,12.079646,NaN,NaN,7.030612,15.449686,NaN,NaN,6.955556,NaN,14.770563,NaN,11.252525,NaN,NaN,6.436937,NaN,7.280637,NaN,NaN,6.131579,NaN,7.681230,9.609053,13.733333,7.807190,11.478964,6.622449,13.475000,7.781481,8.796748,NaN,NaN,14.816327,8.105856,8.086538,15.318519,7.310000,8.848810,NaN,NaN,12.163717,NaN,NaN,10.214286,NaN,16.218750,NaN,12.293103,NaN,NaN,12.878378,NaN,NaN,13.957143,7.941441
3,12.761905,NaN,NaN,9.211712,10.304487,NaN,16.840741,54.063063,NaN,NaN,NaN,NaN,17.622896,NaN,NaN,11.917989,NaN,NaN,NaN,NaN,NaN,25.755556,NaN,NaN,12.891534,NaN,NaN,12.488506,NaN,NaN,8.307598,NaN,NaN,7.425439,NaN,8.930421,13.065844,16.690909,8.862745,13.922330,8.370748,13.100000,13.985185,12.695122,NaN,NaN,32.159864,10.979730,12.512821,22.651852,14.621667,17.594048,NaN,21.649635,NaN,NaN,19.380342,NaN,18.474843,NaN,8.280556,NaN,NaN,8.645503,NaN,NaN,7.683908,NaN,NaN
4,13.309524,NaN,NaN,8.972973,9.147436,NaN,16.385185,52.761261,NaN,NaN,NaN,NaN,15.001684,NaN,NaN,11.082011,NaN,NaN,NaN,NaN,NaN,26.147222,NaN,NaN,10.341270,NaN,NaN,11.948276,NaN,NaN,NaN,NaN,8.311628,NaN,NaN,9.741100,12.781893,18.245455,10.535948,14.281553,9.768707,18.400000,12.948148,9.873984,NaN,NaN,27.299320,12.783784,13.355769,19.796296,10.640000,13.302381,17.264310,NaN,NaN,16.481481,NaN,NaN,22.669811,NaN,10.319444,NaN,16.666667,NaN,NaN,12.292929

In [190]:
pb_top = comb.join(pb_top)

In [191]:
pb_top

,category,gender,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,Masters Men 1 (40-49),Male,NaN,10.019231,6.465116,6.504505,NaN,20.073718,NaN,NaN,6.145000,NaN,8.846429,NaN,NaN,NaN,12.353982,NaN,NaN,7.394558,16.251572,NaN,NaN,8.361111,NaN,16.926407,NaN,10.257576,NaN,NaN,6.599099,NaN,6.627451,NaN,NaN,5.456140,NaN,7.582524,11.248971,14.518182,7.503268,11.203883,15.721088,12.900000,7.118519,9.581301,23.611111,NaN,14.989796,9.288288,8.631410,15.437037,6.750000,9.766667,NaN,NaN,13.104720,NaN,NaN,7.914966,17.150943,NaN,7.433333,NaN,15.958874,NaN,NaN,11.646465,NaN,NaN,8.238739
1,Senior Men (23-39),Male,18.510204,NaN,NaN,11.418919,10.887821,NaN,19.544444,71.972973,NaN,NaN,NaN,NaN,18.698653,NaN,NaN,14.894180,NaN,NaN,NaN,NaN,NaN,31.325000,NaN,NaN,11.539683,NaN,NaN,12.436782,NaN,NaN,NaN,NaN,8.427907,NaN,NaN,10.598706,13.681070,18.960606,12.375817,14.624595,8.982993,17.462500,9.592593,12.430894,NaN,NaN,26.792517,12.295045,13.035256,21.592593,10.710000,13.915476,19.146465,NaN,NaN,17.150794,NaN,NaN,23.805031,NaN,13.816667,NaN,22.911255,NaN,NaN,12.661616,NaN,NaN,10.245495
2,Senior Men (23-39),Male,NaN,10.910256,6.391473,7.560811,NaN,22.368590,NaN,NaN,6.518333,NaN,9.108333,NaN,NaN,NaN,12.079646,NaN,NaN,7.030612,15.449686,NaN,NaN,6.955556,NaN,14.770563,NaN,11.252525,NaN,NaN,6.436937,NaN,7.280637,NaN,NaN,6.131579,NaN,7.681230,9.609053,13.733333,7.807190,11.478964,6.622449,13.475000,7.781481,8.796748,NaN,NaN,14.816327,8.105856,8.086538,15.318519,7.310000,8.848810,NaN,NaN,12.163717,NaN,NaN,10.214286,NaN,16.218750,NaN,12.293103,NaN,NaN,12.878378,NaN,NaN,13.957143,7.941441
3,Senior Men (23-39),Male,12.761905,NaN,NaN,9.211712,10.304487,NaN,16.840741,54.063063,NaN,NaN,NaN,NaN,17.622896,NaN,NaN,11.917989,NaN,NaN,NaN,NaN,NaN,25.755556,NaN,NaN,12.891534,NaN,NaN,12.488506,NaN,NaN,8.307598,NaN,NaN,7.425439,NaN,8.930421,13.065844,16.690909,8.862745,13.922330,8.370748,13.100000,13.985185,12.695122,NaN,NaN,32.159864,10.979730,12.512821,22.651852,14.621667,17.594048,NaN,21.649635,NaN,NaN,19.380342,NaN,18.474843,NaN,8.280556,NaN,NaN,8.645503,NaN,NaN,7.683908,NaN,NaN
4,Senior Men (23-39),Male,13.309524,NaN,NaN,8.972973,9.147436,NaN,16.385185,52.761261,NaN,NaN,NaN,NaN,15.001684,NaN,NaN,11.082011,NaN,NaN,NaN,NaN,NaN,26.147222,NaN,NaN,10.341270,NaN,NaN,11.948276,NaN,NaN,NaN,NaN,8.311628,NaN,NaN,9.741100,12.781893,18.245455,10.535948,14.281553,9.768707,18.400000,12.948148,9.873984,NaN,NaN,27.299320,12

In [192]:
# Read in Split Timings
pb_top.to_csv('../../data/ccc_pb_top.csv', encoding = "ISO-8859-1")

In [193]:
# Read in Split Timings
pb_top = pd.read_csv('../../data/ccc_pb_top.csv', encoding = "ISO-8859-1")

In [194]:
pb_top.iloc[[8]]

,Unnamed: 0,category,gender,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
8,8,Masters Men 1 (40-49),Male,NaN,10.019231,6.465116,6.504505,NaN,20.073718,NaN,NaN,6.145,NaN,8.846429,NaN,NaN,NaN,12.353982,NaN,NaN,7.394558,16.251572,NaN,NaN,8.361111,NaN,16.926407,NaN,10.257576,NaN,NaN,6.599099,NaN,6.627451,NaN,NaN,5.45614,NaN,7.582524,11.248971,14.518182,7.503268,11.203883,15.721088,12.9,7.118519,9.581301,23.611111,NaN,14.989796,9.288288,8.63141,15.437037,6.75,9.766667,NaN,NaN,13.10472,NaN,NaN,7.914966,17.150943,NaN,7.433333,NaN,15.958874,NaN,NaN,11.646465,NaN,NaN,8.238739


# Mean Pace Rec 

In [195]:
def pace_rec(index):
    #print(index)
    query = ccc.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

pace_rec(0)

,Year,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014.4,NaN,12.333333,7.45155,7.872973,8.146368,21.541667,16.574074,NaN,6.893833,NaN,8.794048,NaN,NaN,NaN,12.500147,NaN,NaN,8.062245,17.576101,NaN,NaN,9.2175,NaN,18.278571,NaN,13.612121,NaN,NaN,8.371171,NaN,1102.423333,11.354677,7.893382,NaN,7.452713,6.65692,NaN,8.655663,12.595062,16.236364,8.796732,13.676699,11.287075,17.59125,9.079259,10.518293,37.62963,NaN,22.601361,10.956081,11.455449,22.188519,9.245,11.15631,14.149832,NaN,19.284005,10.828042,NaN,13.64966,20.312893,20.571181,10.059722,19.447318,19.669913,NaN,23.137387,13.883838,NaN,20.244048,11.016216,2212.16,1001.68,14.050342


In [196]:
pb1 = pb.filter(like='to')

In [197]:
pb1.shape

(2798, 39)

In [198]:
mean_pb = pd.DataFrame()

In [199]:
mean_pb

""


In [200]:
pb1.shape[0]

2798

In [201]:
for i in range(pb1.shape[0]):
    mean_pb = mean_pb.append(pace_rec(i), ignore_index=True)

In [202]:
mean_pb

,Year,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014.400000,NaN,12.333333,7.451550,7.872973,8.146368,21.541667,16.574074,NaN,6.893833,NaN,8.794048,NaN,NaN,NaN,12.500147,NaN,NaN,8.062245,17.576101,NaN,NaN,9.217500,NaN,18.278571,NaN,13.612121,NaN,NaN,8.371171,NaN,1102.423333,11.354677,7.893382,NaN,7.452713,6.656920,NaN,8.655663,12.595062,16.236364,8.796732,13.676699,11.287075,17.591250,9.079259,10.518293,37.629630,NaN,22.601361,10.956081,11.455449,22.188519,9.245000,11.156310,14.149832,NaN,19.284005,10.828042,NaN,13.649660,20.312893,20.571181,10.059722,19.447318,19.669913,NaN,23.137387,13.883838,NaN,20.244048,11.016216,2212.160000,1001.680000,14.050342
1,2007.000000,17.090476,NaN,NaN,10.796622,10.998718,NaN,19.455185,67.233333,NaN,NaN,NaN,NaN,19.957744,NaN,NaN,14.752646,NaN,NaN,NaN,NaN,NaN,32.091389,NaN,NaN,13.566667,NaN,NaN,12.703831,NaN,NaN,1236.208333,21.864661,10.116422,7.159058,9.126098,10.043860,29.356667,10.945793,13.981070,18.769394,12.089869,15.541748,9.903061,18.609583,11.041481,14.147154,NaN,NaN,29.247959,14.692342,15.033333,24.017407,12.700333,16.034881,23.008605,16.895377,NaN,18.385655,16.485043,NaN,25.320755,NaN,13.346389,NaN,23.227032,18.394180,NaN,15.607744,17.036398,NaN,12.729730,2649.995000,1384.143333,16.750925
2,2014.500000,NaN,11.453846,6.358527,7.543919,7.636447,21.210470,16.034392,NaN,6.511167,NaN,8.875833,NaN,11.005051,NaN,11.985251,9.678571,NaN,7.513180,16.169811,NaN,NaN,7.018056,NaN,15.564286,NaN,11.737879,NaN,NaN,7.789189,NaN,1018.865000,10.342037,7.789216,NaN,NaN,6.088596,NaN,8.572492,10.608848,15.223636,8.026797,13.172492,11.845238,16.792500,8.872222,10.125203,47.611111,22.726389,21.880466,10.752027,11.987500,20.502222,8.397167,11.656310,20.030303,NaN,17.070141,16.484127,NaN,11.167800,22.703354,18.554688,12.168981,16.105603,19.417749,NaN,18.022523,15.794613,NaN,19.583929,10.009910,2112.838333,898.631667,13.571467
3,2007.600000,13.362434,16.538462,12.449612,9.194820,8.947344,27.964744,17.441799,55.817246,11.165833,14.567647,12.991667,13.600575,16.137566,NaN,18.656342,12.134921,NaN,13.442177,NaN,NaN,46.592147,28.000347,NaN,22.854978,12.811875,16.555556,NaN,12.456790,11.655405,NaN,1132.483333,18.389516,8.401961,4.999275,8.744380,7.779971,22.341667,12.051133,13.891175,20.838182,11.122876,16.587738,12.795918,18.366667,11.602058,13.031707,NaN,NaN,28.788776,13.675425,16.045513,24.498889,12.1

In [203]:
mean_pb = comb.join(mean_pb)

In [204]:
# Read in Split Timings
mean_pb.to_csv('../../data/ccc_mean_pb.csv', encoding = "ISO-8859-1")

In [205]:
# Read in Split Timings
m_pb = pd.read_csv('../../data/ccc_mean_pb.csv', encoding = "ISO-8859-1")